In [1]:
import os, numpy as np, pandas as pd
from math import exp

BASE = os.path.expanduser("~/Desktop/bonds_project")
DATA = os.path.join(BASE, "data", "market_panel.csv")

panel = pd.read_csv(DATA, index_col=0, parse_dates=True)
# Use the most recent date with both yields and ETFs available
mask_cols = ['y_2','y_5','y_10','y_30','IEF','TLT']
asof_idx = panel[mask_cols].dropna().index[-1]
asof = panel.loc[asof_idx]
print("As-of:", asof_idx.date())

# Current ETF prices (used to convert regression betas -> KRD in $ per 1bp)
P_IEF = float(asof['IEF'])
P_TLT = float(asof['TLT'])
print("IEF price:", round(P_IEF, 4), "TLT price:", round(P_TLT, 4))


As-of: 2025-10-20
IEF price: 97.52 TLT price: 91.54


In [2]:
# Curve: treat CMT as zero-rate approximation (course-level simplification)
def build_curve_from_row(row):
    return {
        2.0:  row['y_2']  / 100.0,
        5.0:  row['y_5']  / 100.0,
        10.0: row['y_10'] / 100.0,
        30.0: row['y_30'] / 100.0,
    }

def interp_zero_rate(curve, t_years):
    ks = np.array(sorted(curve.keys()))
    rs = np.array([curve[k] for k in ks])
    t  = float(np.clip(t_years, ks.min(), ks.max()))
    return float(np.interp(t, ks, rs))

def discount_factor_from_curve(curve, t, comp='cont'):
    r = interp_zero_rate(curve, t)
    if comp == 'cont':   return np.exp(-r*t)
    if comp == 'simple': return 1.0/(1.0 + r*t)
    raise ValueError("comp must be 'cont' or 'simple'")

def bond_cashflows(maturity_years, coupon_rate, freq=2, face=100.0):
    n = int(round(maturity_years*freq))
    times = np.arange(1, n+1, dtype=float)/float(freq)
    times[-1] = maturity_years
    c = coupon_rate*face/float(freq)
    cfs = np.full_like(times, c, dtype=float)
    cfs[-1] += face
    return list(zip(times, cfs))

def price_from_curve(curve, T, c, freq=2, face=100.0, comp='cont'):
    pv = 0.0
    for t, cf in bond_cashflows(T, c, freq=freq, face=face):
        pv += cf*discount_factor_from_curve(curve, t, comp=comp)
    return float(pv)

def risk_parallel(curve, T, c, freq=2, face=100.0, comp='cont', bump_bp=1.0):
    bump = bump_bp/10000.0
    p0 = price_from_curve(curve, T, c, freq=freq, face=face, comp=comp)
    up   = {k:v+bump for k,v in curve.items()}
    down = {k:v-bump for k,v in curve.items()}
    pu   = price_from_curve(up,   T, c, freq=freq, face=face, comp=comp)
    pd   = price_from_curve(down, T, c, freq=freq, face=face, comp=comp)
    dur  = - (pu - pd)/(2.0*bump*p0)
    conv = (pu + pd - 2.0*p0)/(p0*(bump**2))
    dv01 = - dur*p0*1e-4
    return p0, float(dur), float(conv), float(dv01)

def krdv01s(curve, T, c, freq=2, face=100.0, comp='cont', bump_bp=1.0, keys=(2.0,5.0,10.0,30.0)):
    P0   = price_from_curve(curve, T, c, freq=freq, face=face, comp=comp)
    bump = bump_bp/10000.0
    out = {}
    for k in keys:
        bumped = dict(curve)
        if k not in bumped:
            bumped[k] = interp_zero_rate(curve, k)
        bumped[k] += bump
        P_up = price_from_curve(bumped, T, c, freq=freq, face=face, comp=comp)
        out[k] = float((P_up - P0)/(-bump_bp))   # per 1 bp
    return out

def bond_spec(T, c, freq=2, face=100.0, qty=1.0):
    return {"T":float(T), "c":float(c), "f":int(freq), "F":float(face), "q":float(qty)}

def portfolio_price(curve, bonds, comp='cont'):
    return float(sum(b["q"]*price_from_curve(curve, b["T"], b["c"], freq=b["f"], face=b["F"], comp=comp) for b in bonds))

def portfolio_parallel_risk(curve, bonds, comp='cont', bump_bp=1.0):
    P0 = portfolio_price(curve, bonds, comp=comp)
    bump = bump_bp/10000.0
    up   = {k:v+bump for k,v in curve.items()}
    down = {k:v-bump for k,v in curve.items()}
    Pu = portfolio_price(up,   bonds, comp=comp)
    Pd = portfolio_price(down, bonds, comp=comp)
    dur  = - (Pu - Pd)/(2.0*bump*P0)
    conv = (Pu + Pd - 2.0*P0)/(P0*(bump**2))
    dv01 = - dur*P0*1e-4
    return P0, float(dur), float(conv), float(dv01)

def portfolio_krdv01s(curve, bonds, comp='cont', bump_bp=1.0, keys=(2.0,5.0,10.0,30.0)):
    P0 = portfolio_price(curve, bonds, comp=comp)
    out = {}
    for k in keys:
        bumped = dict(curve)
        if k not in bumped:
            bumped[k] = interp_zero_rate(curve, k)
        bumped[k] += bump_bp/10000.0
        P_up = portfolio_price(bumped, bonds, comp=comp)
        out[k] = float((P_up - P0)/(-bump_bp))
    return out


In [3]:
# Example: three fixed-coupon Treasuries; edit to your actual portfolio
b1 = bond_spec(2.0, 0.03, freq=2, face=100.0, qty=1.0)   # 2y 3%
b2 = bond_spec(5.0, 0.03, freq=2, face=100.0, qty=1.0)   # 5y 3%
b3 = bond_spec(10.0,0.03, freq=2, face=100.0, qty=1.0)   # 10y 3%
BONDS = [b1,b2,b3]

curve = build_curve_from_row(asof)
P0, D_port, C_port, DV01_port = portfolio_parallel_risk(curve, BONDS)
KRD_port = portfolio_krdv01s(curve, BONDS)

print("Portfolio — Price:", round(P0,4), "Dur:", round(D_port,4), "DV01:", round(DV01_port,6))
print("Portfolio KRDV01s (per bp):", {k: round(v,6) for k,v in KRD_port.items()}, "Sum:", round(sum(KRD_port.values()),6))


Portfolio — Price: 287.9439 Dur: 4.9973 DV01: -0.143893
Portfolio KRDV01s (per bp): {2.0: 0.022913, 5.0: 0.04907, 10.0: 0.071861, 30.0: -0.0} Sum: 0.143844


In [4]:
# Δy in *basis points*; ETF simple returns
df = panel[['y_2','y_5','y_10','y_30','IEF','TLT']].dropna().copy()
for col in ['y_2','y_5','y_10','y_30']:
    df['d_' + col] = df[col].diff() * 100.0   # % → bp
df['ret_IEF'] = df['IEF'].pct_change()
df['ret_TLT'] = df['TLT'].pct_change()
df = df.dropna()

# Restrict to a lookback window (e.g., last 3 years) to keep the regime relevant
cutoff = df.index.max() - pd.DateOffset(years=3)
reg = df.loc[df.index >= cutoff].copy()
print("Regression window:", reg.index.min().date(), "→", reg.index.max().date(), " (n =", len(reg), ")")


Regression window: 2022-10-20 → 2025-10-20  (n = 783 )


In [5]:
def etf_krd_from_regression(reg_df, ticker='IEF', current_price=100.0):
    """
    Regress ETF daily return on Δy_2, Δy_5, Δy_10, Δy_30 (in bp):
        r_t ≈ beta_2 * Δy2_t + beta_5 * Δy5_t + beta_10 * Δy10_t + beta_30 * Δy30_t + const
    Interpretation:
        r ≈ ΔP/P  and  ΔP ≈ - Σ KRD_k * Δbp_k  ⇒  beta_k ≈ -(KRD_k / P)
        => KRD_k ≈ - beta_k * current_price
    Returns a dict of KRDs (per bp, $ per $100 notionally) and summary stats.
    """
    y_cols = ['d_y_2','d_y_5','d_y_10','d_y_30']
    X = reg_df[y_cols].values
    y = reg_df['ret_' + ticker].values
    # add intercept
    X_ = np.column_stack([X, np.ones(len(X))])
    beta, *_ = np.linalg.lstsq(X_, y, rcond=None)  # [b2,b5,b10,b30, c]
    resid = y - X_.dot(beta)
    ss_res = float(np.sum(resid**2))
    ss_tot = float(np.sum((y - y.mean())**2))
    r2 = 1.0 - ss_res/ss_tot if ss_tot > 0 else np.nan

    betas = dict(zip([2.0,5.0,10.0,30.0], beta[:4]))   # per bp (in return units)
    # Convert to KRD (per bp, $ impact per $100 notionally)
    KRD = {k: float(-betas[k] * current_price) for k in betas.keys()}
    DV01 = float(sum(KRD.values()))
    return KRD, DV01, r2, betas

# Estimate for IEF, TLT using current prices at as-of date
KRD_IEF, DV01_IEF, R2_IEF, betas_IEF = etf_krd_from_regression(reg, 'IEF', P_IEF)
KRD_TLT, DV01_TLT, R2_TLT, betas_TLT = etf_krd_from_regression(reg, 'TLT', P_TLT)

print("IEF — DV01:", round(DV01_IEF,6), "KRD:", {k: round(v,6) for k,v in KRD_IEF.items()}, "R^2:", round(R2_IEF,3))
print("TLT — DV01:", round(DV01_TLT,6), "KRD:", {k: round(v,6) for k,v in KRD_TLT.items()}, "R^2:", round(R2_TLT,3))


IEF — DV01: 0.071979 KRD: {2.0: 0.000651, 5.0: 0.018625, 10.0: 0.044757, 30.0: 0.007946} R^2: 0.923
TLT — DV01: 0.152013 KRD: {2.0: 0.002363, 5.0: -0.011528, 10.0: 0.024287, 30.0: 0.13689} R^2: 0.936


In [6]:
def heuristic_etf_krd(effective_duration, current_price, weights):
    """
    Construct a simple KRD split that sums to DV01 = duration * price * 1bp.
    weights: dict mapping {2: w2, 5: w5, 10: w10, 30: w30}, sum to 1.
    Returns {key: KRD_per_bp_in_$} with sum ≈ duration*price*1e-2 (per bp on $100).
    """
    dv01 = effective_duration * current_price * 1e-4   # $ per bp
    krd = {float(k): float(dv01 * w) for k, w in weights.items()}
    return krd, dv01

# Example weights (tweak as you like):
# IEF (7–10y) loads mostly 5y & 10y; TLT (20y+) loads mainly 10y & 30y
heur_IEF, heur_DV01_IEF = heuristic_etf_krd(
    effective_duration=7.5, current_price=P_IEF, weights={2:0.05, 5:0.45, 10:0.45, 30:0.05}
)
heur_TLT, heur_DV01_TLT = heuristic_etf_krd(
    effective_duration=17.0, current_price=P_TLT, weights={2:0.02, 5:0.18, 10:0.40, 30:0.40}
)
print("Heuristic IEF DV01:", round(heur_DV01_IEF,6), "KRD:", {k: round(v,6) for k,v in heur_IEF.items()})
print("Heuristic TLT DV01:", round(heur_DV01_TLT,6), "KRD:", {k: round(v,6) for k,v in heur_TLT.items()})


Heuristic IEF DV01: 0.07314 KRD: {2.0: 0.003657, 5.0: 0.032913, 10.0: 0.032913, 30.0: 0.003657}
Heuristic TLT DV01: 0.155618 KRD: {2.0: 0.003112, 5.0: 0.028011, 10.0: 0.062247, 30.0: 0.062247}


In [7]:
# Choose KRD source:
use_regression = True if (R2_IEF > 0.5 and R2_TLT > 0.5) else False

KRD_I = KRD_IEF if use_regression else heur_IEF
KRD_T = KRD_TLT if use_regression else heur_TLT

# Order keys and build arrays
keys = [2.0,5.0,10.0,30.0]
b_vec = np.array([KRD_port.get(k, 0.0) for k in keys], dtype=float)   # portfolio KRDs (per bp, $ per $100)

A = np.column_stack([
    np.array([KRD_I.get(k, 0.0) for k in keys], dtype=float),  # column for IEF
    np.array([KRD_T.get(k, 0.0) for k in keys], dtype=float),  # column for TLT
])
instruments = ["IEF","TLT"]

print("b (portfolio KRD per bp):", {k: round(v,6) for k,v in zip(keys, b_vec)})
print("A (rows=keys 2/5/10/30; cols=IEF,TLT):\n", pd.DataFrame(A, index=keys, columns=instruments).round(6))


b (portfolio KRD per bp): {2.0: 0.022913, 5.0: 0.04907, 10.0: 0.071861, 30.0: -0.0}
A (rows=keys 2/5/10/30; cols=IEF,TLT):
            IEF       TLT
2.0   0.000651  0.002363
5.0   0.018625 -0.011528
10.0  0.044757  0.024287
30.0  0.007946  0.136890


In [8]:
# Parallel DV01 hedge using TLT by default (long-duration)
# Convention: positive DV01 means long interest-rate risk. To hedge, we SHORT a positive-DV01 instrument.
DV01_port_parallel = float(sum(b_vec))
DV01_TLT_parallel  = float(sum([KRD_T.get(k,0.0) for k in keys]))
h_parallel = DV01_port_parallel / DV01_TLT_parallel  # LONG TLT that much adds DV01; to hedge, SHORT it.

print("Parallel DV01 (portfolio):", round(DV01_port_parallel,6))
print("Parallel DV01 (TLT):      ", round(DV01_TLT_parallel,6))
print("Hedge ratio (TLT) — long units to MATCH DV01:", round(h_parallel,6))
print("Recommended hedge position: SHORT TLT", round(h_parallel,6), "units per $100 portfolio notional")


Parallel DV01 (portfolio): 0.143844
Parallel DV01 (TLT):       0.152013
Hedge ratio (TLT) — long units to MATCH DV01: 0.946262
Recommended hedge position: SHORT TLT 0.946262 units per $100 portfolio notional


In [9]:
# Solve A h ≈ b. A is instrument KRDs; b is portfolio KRD.
# If we *add* h units LONG, resulting KRD is b - A*h to hedge → we want b - A*h ≈ 0
# Hence recommended hedge position is SHORT h units.
h_lstsq, *_ = np.linalg.lstsq(A, b_vec, rcond=None)
residual = b_vec - A.dot(h_lstsq)

print("Least-squares solution (LONG units):", dict(zip(instruments, np.round(h_lstsq,6))))
print("Recommended hedge: SHORT", {inst: round(v,6) for inst,v in zip(instruments, h_lstsq)}, "units")
print("Residual KRD per bp after hedge:", {k: round(v,6) for k,v in zip(keys, residual)})
print("Residual L2 (bp-$):", round(float(np.linalg.norm(residual)),6))
print("Residual sum (≈ parallel DV01 left):", round(float(residual.sum()),6))


Least-squares solution (LONG units): {'IEF': 1.814472, 'TLT': -0.119459}
Recommended hedge: SHORT {'IEF': 1.814472, 'TLT': -0.119459} units
Residual KRD per bp after hedge: {2.0: 0.022014, 5.0: 0.013899, 10.0: -0.006448, 30.0: 0.001934}
Residual L2 (bp-$): 0.026891
Residual sum (≈ parallel DV01 left): 0.0314


In [10]:
def report_afterhedge(curve, bonds, A, h, keys=(2.0,5.0,10.0,30.0)):
    """
    Compare pre- vs post-hedge KRD vector and parallel DV01.
    Treat each instrument unit as $100 notional of the ETF.
    """
    # Pre-hedge:
    P0, D0, C0, DV01_0 = portfolio_parallel_risk(curve, bonds)
    KR0 = portfolio_krdv01s(curve, bonds)

    # Hedge KRD (LONG h). Our recommendation is to SHORT h, so subtract:
    hedge_krd = A.dot(h)  # vector in same units as b_vec
    KR_after = {k: float(KR0.get(k,0.0) - hedge_krd[i]) for i,k in enumerate(keys)}
    DV01_after = float(sum(KR_after.values()))

    # Pretty print
    print("\n=== Risk Report (per $100 notional) ===")
    print("Pre-hedge  DV01:", round(DV01_0,6), "KRD:", {k: round(KR0.get(k,0.0),6) for k in keys})
    print("Post-hedge DV01:", round(DV01_after,6), "KRD:", {k: round(KR_after[k],6) for k in keys})
    print("Residual L2:", round(float(np.linalg.norm(np.array([KR_after[k] for k in keys]))),6))

# Parallel hedge as a vector using only TLT:
A_parallel = np.column_stack([np.zeros(4), np.array([KRD_T.get(k,0.0) for k in keys])])
h_parallel_vec = np.array([0.0, DV01_port_parallel / DV01_TLT_parallel], dtype=float)
report_afterhedge(curve, BONDS, A_parallel, h_parallel_vec, keys=keys)

# Key-rate hedge with IEF + TLT:
report_afterhedge(curve, BONDS, A, h_lstsq, keys=keys)



=== Risk Report (per $100 notional) ===
Pre-hedge  DV01: -0.143893 KRD: {2.0: 0.022913, 5.0: 0.04907, 10.0: 0.071861, 30.0: -0.0}
Post-hedge DV01: 0.0 KRD: {2.0: 0.020677, 5.0: 0.059978, 10.0: 0.048879, 30.0: -0.129534}
Residual L2: 0.152293

=== Risk Report (per $100 notional) ===
Pre-hedge  DV01: -0.143893 KRD: {2.0: 0.022913, 5.0: 0.04907, 10.0: 0.071861, 30.0: -0.0}
Post-hedge DV01: 0.0314 KRD: {2.0: 0.022014, 5.0: 0.013899, 10.0: -0.006448, 30.0: 0.001934}
Residual L2: 0.026891
